<a href="https://colab.research.google.com/github/ModernOctave/Thermal-Image-Human-Detection/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pull Code

In [ ]:
!git clone -b v2 https://github.com/ModernOctave/Thermal-Image-Human-Detection.git
%cd Thermal-Image-Human-Detection

# Download Raw Dataset

In [ ]:
!wget -v https://adas-dataset-v2.flirconservator.com/dataset/full/FLIR_ADAS_v2.zip
!unzip -q FLIR_ADAS_v2.zip -d FLIR_ADAS_v2

# Install Dependancies


In [ ]:
import tensorflow as tf

Run the following if using colab,

In [ ]:
# Import Tensorflow 2.0
%tensorflow_version 2.x

# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
assert len(tf.config.list_physical_devices('GPU')) > 0

# Create datasets

In [ ]:
from preprocessing import label_dataset, split_dataset, DataGen

labeled_dataset = label_dataset(dataset_path="FLIR_ADAS_v2/images_thermal_train/")
train_dataset, test_dataset = split_dataset(labeled_dataset, 0.8, shuffle=True)

train_gen = DataGen(train_dataset, batch_size=32)
test_gen = DataGen(test_dataset, batch_size=32)

# Model


## Create the model

In [ ]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S
from tensorflow.keras.metrics import BinaryAccuracy, AUC, FalsePositives, FalseNegatives

base = EfficientNetV2S(include_top=False)
base.trainable = False

model = tf.keras.Sequential([
    tf.keras.layers.Resizing(384, 384, input_shape=(512, 640, 3)),
    base,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.BinaryCrossentropy(),
             metrics=[BinaryAccuracy(), AUC(), FalsePositives(), FalseNegatives()])

print(model.summary())

## Train the model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [EarlyStopping(patience=2, restore_best_weights=True)]

model.fit(train_gen, epochs=1000, validation_data=test_gen, callbacks=callbacks)

# Load or Save the Model

In [ ]:
model = tf.keras.models.load_model('model')

In [ ]:
model.save('model')

## Test the model


In [ ]:
loss, accuracy = cnn_model.evaluate(test_images, test_labels, batch_size=1)